# Power Outage Data

- Collected from: https://www.oe.netl.doe.gov/OE417_annual_summary.aspx
- Report information: https://www.oe.netl.doe.gov/docs/OE417_Form_Instructions_05312021.pdf




In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import power outage data & concat from year 2014 - 2019
data_list = []
for i in range(4,10,1):
    data = pd.read_excel(f"../Data/201{i}_Annual_Summary.xls", header = 1)
    data_list.append(data)
df = pd.concat(data_list, axis=0, ignore_index=True)

#https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

WARNING *** file size (73412) not 512 + multiple of sector size (512)
WARNING *** file size (73412) not 512 + multiple of sector size (512)
WARNING *** file size (69316) not 512 + multiple of sector size (512)
WARNING *** file size (86212) not 512 + multiple of sector size (512)
WARNING *** file size (79044) not 512 + multiple of sector size (512)


In [3]:
df.isnull().sum()

Month                            0
Date Event Began                 0
Time Event Began                 0
Date of Restoration              0
Time of Restoration              0
Area Affected                    0
NERC Region                      9
Alert Criteria                   0
Event Type                       0
Demand Loss (MW)                18
Number of Customers Affected    18
dtype: int64

In [4]:
# Fill NA with "Unknown"
df = df.fillna("Unknown")

In [5]:
df.shape

(1060, 11)

**Capture state name in the `Area Affected` column**


In [6]:
def get_state(string):
    states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
      "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
      "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
      "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
      "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
      "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
      "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
      "Vermont","Washington","West Virginia","Wisconsin","Wyoming", "District of Columbia", "Unknown"]
    state_list = []
    for i in states:
        if i.title() in string.title():
            state_list.append(i)
    if ("Virginia" in string.title()) & ~("West Virginia" in string.title()):
        state_list.append("Virginia")
    if string.title().count("Virginia") >= 2:
        state_list.append("Virginia")
    return state_list

In [7]:
df['state'] = df['Area Affected'].map(get_state)

**Separate mutiple states to one state per row**

In [9]:
df_test = df.iloc[0:1].copy()
for i in range(1, df.shape[0], 1):
    if len(df['state'][i]) == 1:
        df_test = df_test.append(df.iloc[i:i+1])
        df_test['state'][i] = df['state'][i][0]
    else:
        for j in range(0, len(df['state'][i]), 1):
            df_test = df_test.append(df.iloc[i:i+1])
            df_test.iloc[-1:]['state'] = df['state'][i][j]
df_test['state'][0] = df['state'][0][0]

df_test.shape

/Users/jackwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


(1325, 12)

In [10]:
# Reset index
df_test.reset_index(drop = True, inplace = True)

**Add `time_location` column to the dataframe so we can merge with twitter data**

In [11]:
# Format date column
df_test['Date Event Began'] = pd.to_datetime(df_test['Date Event Began']).dt.date.astype("object")

# Format hour column
df_test['Hour Event Began'] = pd.to_datetime(df_test['Time Event Began'], format ='%H:%M:%S').dt.hour.astype("object")

# Create time_location column
df_test['time_location'] = [f"{df_test['Date Event Began'][i]} {df_test['Hour Event Began'][i]}:00:00 | {df_test['state'][i]}"
 for i in range(0,df_test.shape[0],1)]

In [16]:
df_test.head()

,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected,state,Hour Event Began,time_location
0,January,2014-01-06,19:50:00,01/06/2014,20:44:00,District of Columbia,RFC,Missing,Voltage Reduction due to Severe Weather - Cold,Unknown,Unknown,District of Columbia,19,2014-01-06 19:00:00 | District of Columbia
1,January,2014-01-06,19:50:00,01/06/2014,20:44:00,Unknown,RFC,Missing,Voltage Reduction due to Severe Weather - Cold,Unknown,Unknown,Unknown,19,2014-01-06 19:00:00 | Unknown
2,January,2014-01-06,19:52:00,01/06/2014,20:45:00,Delaware,RFC,Missing,Voltage Reduction due to Severe Weather - Cold,Unknown,Unknown,Delaware,19,2014-01-06 19:00:00 | Delaware
3,January,2014-01-06,20:45:00,01/07/2014,21:00:00,Unknown,RFC,Missing,Public Appeal due to Severe Weather - Cold,Unknown,Unknown,Unknown,20,2014-01-06 20:00:00 | Unknown
4,January,2014-01-06,19:50:00,01/06/2014,20:49:00,Pennsylvania,RFC,Missing,Voltage Reduction due to Severe Weather - Cold,200,62000,Pennsylvania,19,2014-01-06 19:00:00 | Pennsylvania


In [15]:
# Export to CSV
df_test.to_csv("../Data/power_outage.csv")